In [41]:
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import sys
import os
import gzip
import csv
from sklearn.metrics.pairwise import paired_cosine_distances
from scipy.stats import pearsonr

sts_dataset_path = '../datasets/stsbenchmark.tsv.gz'

if not os.path.exists(sts_dataset_path):
    util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)

train_samples = []
dev_samples = []
test_samples = []
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
        inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)

        if row['split'] == 'dev':
            dev_samples.append(inp_example)
        elif row['split'] == 'test':
            test_samples.append(inp_example)
        else:
            train_samples.append(inp_example)


In [11]:
BERT = "bert-base-cased"
ELECTRA = "google/electra-base-discriminator"

In [35]:
words1 = ["animal", "old",  "strong", "looking", "amazing", "awful", "calm", "idea", "new", "part", "right"] 
words2 = ["beast", "aged", "mighty", "facing", "incredible", "dreadful", "quiet", "thought", "novel", "portion", "correct"]

words_text_examples = [InputExample(texts=[w1, w2], label=1.0) for w1, w2 in zip(words1, words2)]

In [19]:
word_embedding_model = models.Transformer(BERT)
pooling_model = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False
)
model_bert = SentenceTransformer(modules=[word_embedding_model, pooling_model])

word_embedding_model2 = models.Transformer(ELECTRA)
pooling_model2 = models.Pooling(
    word_embedding_model.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True,
    pooling_mode_cls_token=False,
    pooling_mode_max_tokens=False
)
model_electra = SentenceTransformer(modules=[word_embedding_model2, pooling_model2])

In [22]:
bert_enc_1 = model_bert.encode(words1, convert_to_tensor=True)
bert_enc_2 = model_bert.encode(words2, convert_to_tensor=True)

electra_enc_1 = model_electra.encode(words1, convert_to_tensor=True)
electra_enc_2 = model_electra.encode(words2, convert_to_tensor=True)

bert_scores = util.cos_sim(bert_enc_1, bert_enc_2)
electra_scores = util.cos_sim(electra_enc_1, electra_enc_2)

In [47]:
for i in range(len(bert_scores)):
    print("{} {} \t\t Score: {:.4f}".format(words1[i], words2[i], bert_scores[i][i]))

animal beast 		 Score: 0.9135
old aged 		 Score: 0.8663
strong mighty 		 Score: 0.8625
looking facing 		 Score: 0.8844
amazing incredible 		 Score: 0.9700
awful dreadful 		 Score: 0.9701
calm quiet 		 Score: 0.9248
idea thought 		 Score: 0.7952
new novel 		 Score: 0.8090
part portion 		 Score: 0.9058
right correct 		 Score: 0.9229


In [31]:
for i in range(len(electra_scores)):
    print("{} {} \t\t Score: {:.4f}".format(words1[i], words2[i], electra_scores[i][i]))

animal beast 		 Score: 0.7229
old aged 		 Score: 0.5902
strong mighty 		 Score: 0.3471
looking facing 		 Score: 0.5824
amazing incredible 		 Score: 0.9598
awful dreadful 		 Score: 0.8821
calm quiet 		 Score: 0.7524
idea thought 		 Score: 0.6633
new novel 		 Score: 0.5860
part portion 		 Score: 0.7643
right correct 		 Score: 0.8171
